In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
import sys
sys.path.append('../lib')
import shane
import data_utils
import mint_v2
import token_logic_modules as tlm

# Comparison of Shane's model, MINT-V2 and TLMs

This is a simple re-execution of [Shane's final tokenomics spreadsheet](https://docs.google.com/spreadsheets/d/1-_G8VCQ7WbytNmps_N6LhQJatPN3CvV6z2_e0bN9cC0/edit?usp=sharing) and a side-by-side execution of the MINT-V2 and TLMs code.

This is just an static test amid to be used only to compare the models against the spreadsheet calculation.

# Parameters

In [2]:
# Download "Performance" card from https://poktscan.com/explore?tab=chains
CSV_FILE = '../data/chains_performance_2024-05-20_SAMPLE.csv'

# Network Conditions
network_macro = dict()
network_macro['POKT_value'] = 0.50               # USD/POKT
network_macro['total_supply'] = 1679901301.43    # POKT
network_macro['POKT_stake_per_node'] = 60e3      # POKT (irrelevant but for APR calculation, which is a non-goal)
network_macro['relay_multiplier'] = 1            # TODO: WHAT IS THIS SUPPOSED TO BE?
network_macro['base_CU_per_relay'] = 100        # CUs/relay
network_macro['cu_cost'] = 0.0000000085 # USD/CU
network_macro['mint_share'] = {
                                    "DAO"       : 0.10,
                                    "Validator" : 0.05,
                                    "Supplier"   : 0.7,
                                    "Source"    : 0.15,
                                }

assert sum(network_macro['mint_share'].values()) == 1, "The mint share must sum 1 (100%%)"

### Read Chains Data

In [3]:
# Read chains data
chains_df = data_utils.get_services_df(
    CSV_FILE, relay_multiplier=network_macro["relay_multiplier"], spreadsheet_compact=True
)

# Add compute cost
chains_df = data_utils.add_compute_cost(chains_df, sample_type="uniform", mean=network_macro["base_CU_per_relay"])

# To copy spreadsheet, set the solana custom to twice the base cost
chains_df.loc[chains_df['Chain'] == 'Solana Custom (C006)', 'cu_per_relay'] = network_macro['base_CU_per_relay']*2

Index(['Chain', 'Earn AVG', 'Network', 'Change', 'relays', 'active_nodes'], dtype='object')


In [5]:
# From data
network_macro['daily_relays'] = chains_df['relays'].sum()
network_macro['supplier_nodes'] = chains_df['active_nodes'].sum()
network_macro

{'POKT_value': 0.5,
 'total_supply': 1679901301.43,
 'POKT_stake_per_node': 60000.0,
 'relay_multiplier': 1,
 'base_CU_per_relay': 100,
 'cu_cost': 8.5e-09,
 'mint_share': {'DAO': 0.1,
  'Validator': 0.05,
  'Supplier': 0.7,
  'Source': 0.15},
 'daily_relays': 512183851.0,
 'supplier_nodes': 198390.0}

### Shane's Parameters

In [6]:
shane_params = dict()

# Supplier Boost
shane_params['deflation_threshold'] = 25e9          # relays by day
shane_params['max_mint_per_day'] = 180e3            # POKT

# Sources Boost
shane_params['cutoff_point'] = 20e9                 # relays/day
shane_params['prop_of_CUF'] = 0.7                   # -

# Miscellaneous (Sources Boost)
shane_params['prop_ceiling'] = 0.0                  # -
shane_params['floor_relays'] = 5e9                  # relays by day
shane_params['gfpr_rar_ceiling_relays'] = 1.5e9     # relays by day
shane_params['gfpr_rar_prop_ceiling'] = 0.15        # -
shane_params['gfpr_rar_floor_relays'] = 5e6         # relays by day
shane_params['gfpr_rar_prop_floor'] = 0.9           # -
shane_params['gfpr_rar_max_supplier_boost'] = 40e3  # USD


### MINT-V2 Parameters

In [7]:
mint_v2_params = deepcopy(mint_v2.data_dict_defaults)

# Ecosystem Costs and Charges
mint_v2_params["MaturityComputeUnitsCharge"]                = 0.85e-8       # USD/ComputeUnits
mint_v2_params["MaturityComputeUnitsCost"]                  = 0.85e-8       # USD/ComputeUnits
mint_v2_params["SupplyGrowCap"]                             = 0.05          # times

# Servicers
mint_v2_params["MaxBootstrapServicerCostPerComputeUnits"]   = 5e-8          # USD/ComputeUnits
mint_v2_params["ServicersBootstrapUnwindStart"]             = 250           # Billon ComputeUnits/day
mint_v2_params["ServicersBootstrapEnd"]                     = 2500          # Billon ComputeUnits/day
mint_v2_params["MinUsdMint"]                                = 20e3          # USD/day

# Gateways
mint_v2_params["MinBootstrapGatewayFeePerComputeUnits"]     = 0.85e-8       # USD/ComputeUnits
mint_v2_params["GatewaysBootstrapUnwindStart"]              = 250           # Billon ComputeUnits/day
mint_v2_params["GatewaysBootstrapEnd"]                      = 2500          # Billon ComputeUnits/day

# Check that the minimum minting is not higher than the maturity (miny <= burn) phase minting
assert mint_v2_params["MinUsdMint"] <= mint_v2_params["MaturityComputeUnitsCharge"]*mint_v2_params["GatewaysBootstrapEnd"]*1e9


# Additional parameters to activate entropy (just for test)
mint_v2_params['apply_entropy'] = False


### Token Logic Modules (TLMs) Parameters

This is only a simple override, please see all parameters in the module file.

In [8]:
tlm_params = deepcopy(tlm.global_params_dict)
tlm_params['core_TLM']['cu_cost'] = network_macro['cu_cost']
tlm_params['apply_entropy'] = mint_v2_params['apply_entropy']

# Compare

In [9]:
# Process Shane
shanes_per_service_df, shanes_results  = shane.process(chains_df, network_macro, shane_params)
# Process MINT-V2
mint_v2_per_service_df, mint_v2_results  = mint_v2.process(chains_df, network_macro, mint_v2_params)
# Process TLM
tlm_per_service_df, tlm_results  = tlm.process(chains_df, network_macro, tlm_params)

In [12]:
# Comparison dataframe on the spreadsheet highlighted results (in green)
data_rows = list()

for actor, results in zip(['Shane', 'MINT-V2', 'TLM'], [shanes_results, mint_v2_results, tlm_results]):
    row_values = list()
    row_values.append(actor)
    # Supply growth rate
    row_values.append(((results['total_mint'] - results['total_burn'])*365.25/network_macro['total_supply'])*100.)
    # Over-minted total
    row_values.append(results['total_mint']-results['total_burn'])
    # POKT burnt
    row_values.append(results['total_burn'])
    # Supplier monthly income in USD
    month_income_usd = (
        # results["total_mint_base"]
        # * network_macro["mint_share"]["Supplier"]
        results['total_mint_supplier']
        * 30.4
        * network_macro["POKT_value"]
        / network_macro["supplier_nodes"]
    )
    row_values.append(month_income_usd)
    # Supplier APR
    row_values.append(((month_income_usd/30.4)*365./(network_macro['POKT_stake_per_node']*network_macro['POKT_value']))*100.)
    # Over-mint per day due to suppliers extra minting
    row_values.append(results['total_mint_supplier']-results['total_burn'] * network_macro["mint_share"]["Supplier"])
    # Over-minted to sources
    row_values.append(results['total_mint_source']-results['total_burn'] * network_macro["mint_share"]["Source"])

    data_rows.append(row_values)

results_df = pd.DataFrame(
    data_rows,
    columns=[
        "Model",
        "Supply Growth Rate [%]",
        "Over-minting per Day [POKT]",
        "POKT burnt per Day [POKT]",
        "Supplier Income per Day [USD]",
        "Supplier APR (USD) [%]",
        "Over-mint per Day for Suppliers [POKT]",
        "Over-mint per Day for Sources [POKT]",
    ],
)

In [13]:
results_df

,Model,Supply Growth Rate [%],Over-minting per Day [POKT],POKT burnt per Day [POKT],Supplier Income per Day [USD],Supplier APR (USD) [%],Over-mint per Day for Suppliers [POKT],Over-mint per Day for Sources [POKT]
0,Shane,1.036518,47672.750233,984.486454,3.118126,0.124793,40008.553101,603.864233
1,MINT-V2,0.848289,39015.513546,984.486454,2.145269,0.085858,27310.859482,5852.327032
2,TLM,0.084361,3880.034849,984.486454,0.260893,0.010441,2716.024395,582.005227


(Note the lower values of TLM for not aplying minimum mint)